# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import functions_notebook as fn
import costants as cs
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data_cc_30m = fn.open_convert_capture(".\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json")
data_cc_1h = fn.open_convert_capture(".\\captures\\captures1_v2\\clean\\eth2dump-clean-1h_1.json")

data_pf_att_1m_30m_1 = fn.open_convert_capture(".\\captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_2 = fn.open_convert_capture(".\\captures\\captures2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_3 = fn.open_convert_capture(".\\captures\\captures3\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")

In [3]:
pv_cc_30m = fn.create_array_values(data_cc_30m)
pv_cc_1h = fn.create_array_values(data_cc_1h)

pv_pf_att_1m_30m_1 = fn.create_array_values(data_pf_att_1m_30m_1)
pv_pf_att_1m_30m_2 = fn.create_array_values(data_pf_att_1m_30m_2)
pv_pf_att_1m_30m_3 = fn.create_array_values(data_pf_att_1m_30m_3)

In [4]:
# Costruzione dataframe
# Conversione colonna timestamp a float
# Inserimento colonna inter packet arrival time
# Cattura clean 30m
df_cc_30m = pd.DataFrame(data=pv_cc_30m, columns=cs.df_columns)
df_cc_30m['frame.time_epoch'] = df_cc_30m['frame.time_epoch'].astype('float')
df_cc_30m[cs.ipat_field_name] = df_cc_30m['frame.time_epoch'].diff()

# Cattura clean 1h
df_cc_1h = pd.DataFrame(data=pv_cc_1h, columns=cs.df_columns)
df_cc_1h['frame.time_epoch'] = df_cc_1h['frame.time_epoch'].astype('float')
df_cc_1h[cs.ipat_field_name] = df_cc_1h['frame.time_epoch'].diff()

# Catture pf att 1m 30m
df1_pfatt_1m_30m_1 = pd.DataFrame(pv_pf_att_1m_30m_1, columns=cs.df_columns)
df2_pfatt_1m_30m_2 = pd.DataFrame(pv_pf_att_1m_30m_2, columns=cs.df_columns)
df3_pfatt_1m_30m_3 = pd.DataFrame(pv_pf_att_1m_30m_3, columns=cs.df_columns)

# Concatenazione per righe
df_pfatt_1m_30m = pd.concat([df1_pfatt_1m_30m_1, df2_pfatt_1m_30m_2, df3_pfatt_1m_30m_3], ignore_index=True)

In [5]:
df_cc_30m = fn.clean_df(df_cc_30m)
df_cc_1h = fn.clean_df(df_cc_1h)
df_pfatt_1m_30m = fn.clean_df(df_pfatt_1m_30m)

In [6]:
df_pfatt_1m_30m["eth.dst_int"]

0           553850065211
1         79556642095225
2         79556642095225
3           553850065211
4         79556642095225
               ...      
124230       52237803166
124231      553850065211
124232      553850065211
124233       52237803166
124234      553850065211
Name: eth.dst_int, Length: 124235, dtype: int64